# How Does It Work?

To solve a differential equation. We need the solution to satisfy 2 things:
1. They need to satisfy the equation
2. They need to satisfy the initial/boundary conditions

## The Equation Part

The first part is done by 'minimizing' the equation. For example, if we are solving:

$$
\frac{dx}{dt} - x = 0
$$

and we choose L2-loss as our loss function, then the objective of out optimization problem will be:

$$
\min_{\vec{p}}(\frac{d\tilde{x}(\vec{p}, t)}{dt} - \tilde{x}(\vec{p}, t))^2
$$

Where $\vec{p}$ are the weights of the neural network and $\tilde{x(\vec{p}, t)}$ is the modified output of the neural network. How it is modified will be explained in the next section. We can see that when this objective function is driven to 0, then the equation is satisfied. 

## The Initial/Boundary Conditions Part

To solve an ODE, our neural network will have 1 input unit and 1 output unit to approximate an $\mathbb{R} \to \mathbb{R}$ function. Suppose the output of the neural network is $x(\vec{p}, t)$. The output of the neural network won't satisfy the initial/boundary condition unless we enforce the form of our solution. For example, for a initial value problem where we want $x(t)\bigg|_{t = t_0} = x_0$. We can write our solution in the form of

$$
\tilde{x}(\vec{p}, t) = (1-e^{t_0-t})x(\vec{p}, t) + x_0
$$

In this way, when $t = t_0$, the first term will vanish and the value of $\tilde{x}(\vec{p}, t)$ is bound to be $x_0$. In `neurodiffeq`, the output of the solution first goes through this transformation before the objective function in the previous section is calculated.

This trick can be used on other initial/boundary conditions as well.